In [30]:
import random
import numpy as np
pixel_depth=3

lowerBound=0
upperBound=pow(2,pixel_depth-1)

pix1=random.randint(lowerBound,upperBound)
pix2=random.randint(lowerBound,upperBound)
pix1_bin=bin(pix1)[2:]
pix2_bin=bin(pix2)[2:]
pix1_bin_padded = list(reversed('0'*(pixel_depth-len(pix1_bin))+pix1_bin))
pix2_bin_padded = list(reversed('0'*(pixel_depth-len(pix2_bin))+pix2_bin))

  #interleave pix1 and pix2
input_seq_bin = pix1_bin_padded + pix2_bin_padded
input_seq_bin[::2] = list(reversed(pix1_bin_padded))
input_seq_bin[1::2] = list(reversed(pix2_bin_padded))
output_seq_bin = pix1_bin_padded + pix2_bin_padded

input_seq_bin = np.array(input_seq_bin, dtype=np.int)
output_seq_bin = np.array(output_seq_bin, dtype=np.int)

testFlag=1
if testFlag == 1:
    print('pix1 dig: {}, bin: {}'.format(pix1, pix1_bin_padded))
    print('pix2 dig: {}, bin: {}'.format(pix2, pix2_bin_padded))
    print('input seq: {}'.format(input_seq_bin))
    print('output seq: {}'.format(output_seq_bin))

pix1 dig: 2, bin: ['0', '1', '0']
pix2 dig: 3, bin: ['1', '1', '0']
input seq: [0 0 1 1 0 1]
output seq: [0 1 0 1 1 0]


In [35]:
from __future__ import print_function
import numpy as np
from time import sleep
import random
import sys
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
random.seed(10)

def getSample(pixel_depth, testFlag):
    lowerBound=0
    upperBound=pow(2,pixel_depth-1)

    pix1=random.randint(lowerBound,upperBound)
    pix2=random.randint(lowerBound,upperBound)
    pix1_bin=bin(pix1)[2:]
    pix2_bin=bin(pix2)[2:]
    pix1_bin_padded = list(reversed('0'*(pixel_depth-len(pix1_bin))+pix1_bin))
    pix2_bin_padded = list(reversed('0'*(pixel_depth-len(pix2_bin))+pix2_bin))

    #interleave pix1 and pix2, MSB first
    input_seq_bin = pix1_bin_padded + pix2_bin_padded
    input_seq_bin[::2] = list(reversed(pix1_bin_padded))
    input_seq_bin[1::2] = list(reversed(pix2_bin_padded))

    output_seq_bin = pix1_bin_padded + pix2_bin_padded

    #cast output to numpy array
    input_seq_bin = np.array(input_seq_bin, dtype=np.int)
    output_seq_bin = np.array(output_seq_bin, dtype=np.int)

    if testFlag == 1:
        print('pix1 dig: {}, bin: {}'.format(pix1, pix1_bin_padded))
        print('pix2 dig: {}, bin: {}'.format(pix2, pix2_bin_padded))
        print('input seq: {}'.format(input_seq_bin))
        print('output seq: {}'.format(output_seq_bin))

    return input_seq_bin, output_seq_bin


class Model(nn.Module):
  def __init__(self, inputDim, hiddenDim, outputDim):
    super(Model, self).__init__()
    self.inputDim = inputDim
    self.hiddenDim = hiddenDim
    self.outputDim = outputDim
    self.rnn = nn.RNN(inputDim, hiddenDim)
    self.outputLayer = nn.Linear(hiddenDim, outputDim)
    self.sigmoid = nn.Sigmoid()
  def forward(self, x):
    #size of x is T x B x featDim
    #B = 1 is dummy batch dimension added, because pytorch mandates it
    #if you want B as first dimension of x then specify batchFirst=True when LSTM is initalized
    #T,D  = x.size(0), x.size(1)
    #batch is a must
    out,hidden = self.rnn(x.unsqueeze(1)) #x has two  dimensions  seqLen *batch* FeatDim=2
    T,B,D  = out.size(0), out.size(1), out.size(2)
    out = out.contiguous()
    out = out.view(B*T, D)
    outputLayerActivations = self.outputLayer(out)
    outputSigmoid = self.sigmoid(outputLayerActivations)
    return outputSigmoid

inputDim = 1 # two bits each from each of the String
outputDim = 1 # one output node which would output a zero or 1

rnnSize=10

lossFunction = nn.MSELoss()
model = Model(inputDim, rnnSize, outputDim)
optimizer=optim.Adam(model.parameters(),lr=0.001)
epochs=10000
totalLoss= float("inf")

print(" Avg. Loss for last 500 samples = %lf"%(totalLoss))
totalLoss=0
for i in range(0,epochs): # average the loss over 200 samples
    stringLen=4
    testFlag=0
    x,y = getSample(stringLen, testFlag)

    model.zero_grad()

    x_var=autograd.Variable(torch.from_numpy(x).unsqueeze(1).float()) #convert to torch tensor and variable
    # unsqueeze() is used to add the extra dimension since
    # your input need to be of t*batchsize*featDim; you cant do away with the batch in pytorch
    seqLen = x_var.size(0)
    x_var = x_var.contiguous()
    y_var = autograd.Variable(torch.from_numpy(y).float())
    finalScores = model(x_var)

    loss = lossFunction(finalScores,y_var)
    totalLoss+=loss.data
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

totalLoss = totalLoss/epochs
print('Final total loss is:' + str(totalLoss))

 Avg. Loss for last 500 samples = inf
Final total loss is:tensor(0.2241)


In [13]:
###### Testing the model ######

pixel_depth = 3
testFlag=1

for i in range (0,10):
	x,y=getSample(pixel_depth,testFlag)
	x_var=autograd.Variable(torch.from_numpy(x).unsqueeze(1).float())
	y_var=autograd.Variable(torch.from_numpy(y).float())
	seqLen=x_var.size(0)
	x_var= x_var.contiguous()
	finalScores = model(x_var).data.t()
	#print(finalScores)
	bits=finalScores.gt(0.5)
	bits=bits[0].numpy()


input numbers and their sum  are 25   33   58
binary strings are 11001   100001   111010
input numbers and their sum  are 108   78   186
binary strings are 1101100   1001110   10111010
input numbers and their sum  are 12   56   68
binary strings are 1100   111000   1000100
input numbers and their sum  are 108   80   188
binary strings are 1101100   1010000   10111100
input numbers and their sum  are 72   112   184
binary strings are 1001000   1110000   10111000
input numbers and their sum  are 110   5   115
binary strings are 1101110   101   1110011
input numbers and their sum  are 55   41   96
binary strings are 110111   101001   1100000
input numbers and their sum  are 53   18   71
binary strings are 110101   10010   1000111
input numbers and their sum  are 8   36   44
binary strings are 1000   100100   101100
input numbers and their sum  are 69   82   151
binary strings are 1000101   1010010   10010111


In [36]:
###### Testing the model ######

stringLen=7
testFlag=1
# test the network on 10 random binary string addition cases where stringLen=4
for i in range (0,10):
	x,y=getSample(stringLen,testFlag)
	x_var=autograd.Variable(torch.from_numpy(x).unsqueeze(1).float())
	y_var=autograd.Variable(torch.from_numpy(y).float())
	seqLen=x_var.size(0)
	x_var= x_var.contiguous()
	finalScores = model(x_var).data.t()
	#print(finalScores)
	bits=finalScores.gt(0.5)
	bits=bits[0].numpy()

	print ('sum predicted by RNN is ',bits[::-1])
	print('##################################################')


pix1 dig: 46, bin: ['0', '1', '1', '1', '0', '1', '0']
pix2 dig: 47, bin: ['1', '1', '1', '1', '0', '1', '0']
input seq: [0 0 1 1 0 0 1 1 1 1 1 1 0 1]
output seq: [0 1 1 1 0 1 0 1 1 1 1 0 1 0]
sum predicted by RNN is  [ True  True  True  True  True  True False False False False False False
 False False]
##################################################
pix1 dig: 59, bin: ['1', '1', '0', '1', '1', '1', '0']
pix2 dig: 63, bin: ['1', '1', '1', '1', '1', '1', '0']
input seq: [0 0 1 1 1 1 1 1 0 1 1 1 1 1]
output seq: [1 1 0 1 1 1 0 1 1 1 1 1 1 0]
sum predicted by RNN is  [ True  True  True  True  True  True  True  True  True  True False False
 False False]
##################################################
pix1 dig: 64, bin: ['0', '0', '0', '0', '0', '0', '1']
pix2 dig: 19, bin: ['1', '1', '0', '0', '1', '0', '0']
input seq: [1 0 0 0 0 1 0 0 0 0 0 1 0 1]
output seq: [0 0 0 0 0 0 1 1 1 0 0 1 0 0]
sum predicted by RNN is  [False False False False False False False False False False False Fal